In [ ]:


import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv("azure.env")


client = AzureOpenAI(
  azure_endpoint = os.getenv("OPENAI_API_BASE"),
  api_key=os.getenv("OPENAI_API_KEY"),  
  api_version="2024-02-15-preview"
)

In [ ]:
def generate_response(prompt):

    deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
    message_text = [{"role":"system","content": system_prompt},{"role":"user","content":prompt}]

    chat_completion = client.chat.completions.create(
    model=deployment_name,
    messages = message_text,
    temperature=0.7,
    max_tokens=2048,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
    )
  
    message = chat_completion.choices[0].message.content

    return message

In [ ]:


system_prompt ='''
As an Amazon listing optimization expert, the following content provides guidance on optimizing the title and bullet points for an Amazon listing.

For the Title:

	1. Incorporate Essential Details: The title of the product should encapsulate key information. 
       This typically encompasses the brand name, product type, material, key features, size, and quantity.
	2. Utilize High-Ranking Keywords: Implement crucial and highly-ranked keywords in the title. 
       These should correspond to terms that a user might enter when searching for products.
	3. Maintain Clarity and Brevity: Although Amazon permits up to 200 characters for a product title. 
       It's advisable to keep it as clear and succinct as possible. Avoid misleading titles or keyword stuffing.
	4. Refrain from Promotional Language: Exclude promotional phrases such as "top-selling", "best product", etc..
       As Amazon may suppress your listing due to this.
	5. Capitalize the Initial Letter of Each Word: This enhances readability.
	
For Bullet Points:
	1. Emphasize Key Features: Utilize bullet points to spotlight essential features and benefits. 
       These could include dimensions, age appropriateness, ideal usage conditions, and more.
	2. Implement High-Ranking Keywords: Seamlessly integrate relevant and high-ranking keywords within your bullet points to improve visibility.
	3. Keep it Simple and Concise: Maintain simplicity and brevity as customers typically skim through the content. 
       Ideally, each bullet point should be restricted to a single line.
	4. Incorporate Numerical Data: Product specifications can be more effectively communicated using numbers - this often enhances readability and comprehension.
	5. Avoid Acronyms: Acronyms can potentially confuse customers. Unless it's a widely accepted term, opt for the full word. 
       Refrain from making any claims or guarantees that cannot be fulfilled.

'''

In [ ]:
def load_file(filename) -> str:
    with  open(filename,encoding='UTF-8') as f:
        content = f.read()
        #print(content)
        f.close()
        return content

In [ ]:
def customer_feedback_keywords():

    feedback = load_file("data/customer_feedback.txt")

    summary_prompt =f''' 
    user:
    Text content: {feedback}
    
    1. Please scrutinize the above reviews and distill the product's advantages and highlights. 
    2. Condense these benefits and highlights into keywords, focusing on the top 20 most significant ones. 
    3. List the keywords directly, with no need for numbering or sequencing. 
    4. Each keyword should be on a separate line.    
    '''
 
    customer_feedback_keywords = generate_response(summary_prompt)
    print(customer_feedback_keywords)

In [ ]:
customer_feedback_keywords = customer_feedback_keywords()

In [ ]:
def product_highlight():

    key_words = load_file("data/key_words.txt")

    summary_prompt =f''' 
    user:
    Text content: {key_words}
    
    Please analyze and summarize the advantages and highlights of the product based on the above reviews. Summarize the advantages and highlights into keywords, just need the top 20 most important keywords.List keywords directly, no numbering or sequence required, one per line. 使用中文输出结果，按行输出每个关键词.
    '''
    print(summary_prompt)

   
    summary = generate_response(summary_prompt)
    print(summary)

In [ ]:
product_highlight()

In [ ]:
def keywords_from_search():

    keywords_from_search = load_file("data/search_words.txt")

    prohibited_words = load_file("data/prohibited_words.txt")

    summary_prompt =f''' 
    user:
    search terms: 
    {keywords_from_search}

    Prohibited words: 
    {prohibited_words}
    
    1. Based on the user's search terms, please summarize the context and extract the main keywords. 
    Be sure to carefully analyze the search terms to identify the most relevant keywords that accurately reflect the user's intent. 
    Do not include any prohibited words.
    2. Once you have a list of potential keywords, please prioritize them based on their relevance and frequency of appearance 
    in the search terms. Only output the top 20 most important keywords.
    3. Additionally, please consider the synonyms and related terms of these keywords to ensure a broad coverage of potential search queries.
    However, these synonyms and related terms should also adhere to the rule of not including any prohibited words and should not exceed 
    the limit of the top 20 keywords.
    4. Only output 20 keywords, no other information is needed.
    
    '''
    
    keywords_from_search = generate_response(summary_prompt)
    print(keywords_from_search)

In [ ]:
keywords_from_search = keywords_from_search()

In [ ]:
def generate_title():
    product_title = load_file("data/title.txt")
    required_words = load_file("data/key_words.txt")

    summary_prompt =f''' 
    user:
    [Product Title]  
    {product_title}
    
    [keywords_from_search]
    {keywords_from_search}
    
    [customer feeback keywords]
    {customer_feedback_keywords}
    
    [required words]
    {required_words}
     
    Please generate a title according to the following requirements:
    1. Preserve the First Half of the Title: The first step is to ensure that the first half of the title 
    (up to the first comma, not within parentheses) remains unchanged. 
    2. Analyze Keywords and Customer Feedback: analyze the keywords from user search terms and customer feedback. 
    Look for common themes and relevant terms. Choose suitable ones to add to the second half of the title. 
    Be sure to avoid using keywords that have repetitive concepts to ensure the title remains clear and concise.
    3. Include Required Words: The title must contain the words listed in the required words section. 
    These words are crucial for ensuring the listing is accurately represented and can be found by users.
    4. Use Correct Grammar: The title should be grammatically correct and read smoothly. 
    This not only makes it easier for users to understand but also gives a professional impression of the listing.
    5. Adhere to Character Limit: The total number of characters in the title must be between 80-120. 
    6. Generate Multiple Titles: Generate 5 different titles following the above guidelines. 
    Each title should be unique and offer a different combination of keywords and phrases.
    7. Choose the Best Title: Finally, review the 5 generated titles and choose the best one. 
    This should be the title that best represents the listing, includes the required words, and is most likely to attract users.

    Remember, only provide the best title, do not use double quotes. The generated best title is:
      
    '''
     
    title = generate_response(summary_prompt)
    print(title)

In [ ]:
generate_title()

In [ ]:
def customer_feedback_summary():

    feedback = load_file("data/customer_feedback.txt")

    summary_prompt =f''' 
    user:
    [Customer Review]:
    {feedback}

    As an AI expert, here is an expanded version of your prompt for analyzing customer reviews and identifying product advantages 
    and highlights for future AWS listing optimization:
    1. Analyze Customer Reviews: Begin by thoroughly examining the customer reviews for the product. 
    Look for common themes, repeated compliments, and unique attributes that customers appreciate. 
    This could include aspects such as performance, usability, unique features, affordability, or customer service.
    2. Identify Advantages: Identify the advantages of the product based on the customer reviews. 
    These are the positive aspects that make the product stand out and are appreciated by customers. 
    They might be related to the product's functionality, design, price, or other attributes.
    3. Highlight Product Highlights: pinpoint the product's highlights, 
    which are its standout features or aspects that customers particularly enjoy or find beneficial. 
    These could be innovative features, exceptional performance, excellent value for money, 
    or anything else that is frequently mentioned in a positive light in the reviews.
    4. List Advantages and Highlights: Once you've identified the advantages and highlights, list them directly, with one per line. 
    There's no need for numbering or sequencing; the goal is to provide a clear and concise summary of the product's strengths 
    based on customer feedback.
    5. Output the results in Chinese.
    '''
    
    customer_feedback_summary = generate_response(summary_prompt)
    print(customer_feedback_summary)

In [ ]:
customer_feedback_summary()

In [ ]:
def generate_bulletin_points():

    product_title = load_file("data/title.txt")
    product_bullet_points = load_file("data/bullet_points.txt")
    prohibited_words = load_file("data/prohibited_words.txt")
    required_words = load_file("data/key_words.txt")

    summary_prompt =f''' 
    user:
    [Product Bullet Points]
    {product_bullet_points}

    [Product Title] 
    {product_title}
    
    [Customer Feedback Summary]
    {customer_feedback_summary}
    
    [Forbidden Words]
    {prohibited_words}

    [Search Keywords ]
    {keywords_from_search}

    [Product Key Features]
    {required_words}
    
    Please optimize product bullet points using professional language style as Required.
    [Required]
        - Review and subtly modify the existing product bullet points.
        - Endeavor to incorporate as many Product Key Features into the bullet points as possible.
        - Please take into account Consumer Concerns.
        - The use of Forbidden Words is strictly prohibited.
        - Ensure the total character count falls within the range of 600 to 700.
        - Maintain the original number of Product Bullet Points; do not add or remove any.
        - Refrain from using full-width characters.
        - Adhere to the correct grammar format and punctuation standards.
        - Generate six lines of key text records.
    Optimized Product Bullet Points are:
    '''
   
    bulletin_points = generate_response(summary_prompt)
    print(bulletin_points)

In [ ]:
generate_bulletin_points()